In [1]:
//val dataset_df = spark.read.option("header","true").csv("Netflix.csv")
val emp = Seq((1,"Smith",1,"10",3000),
              (1,"Smith",1,"10",3000),
    (2,"Rose",1,"20",4000),
    (3,"Williams",1,"10",1000),
    (4,"Jones",2,"10",2000),
    (5,"Brown",2,"40",-1),
    (6,"Brown",2,"50",-1)
  )
val empColumns = Seq("emp_id","name","superior_emp_id","emp_dept_id","salary")
import spark.sqlContext.implicits._
val datasetDF = emp.toDF(empColumns:_*)
datasetDF.show(false)

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.1.9:4040
SparkContext available as 'sc' (version = 3.2.1, master = local[*], app id = local-1653642828563)
SparkSession available as 'spark'


+------+--------+---------------+-----------+------+
|emp_id|name    |superior_emp_id|emp_dept_id|salary|
+------+--------+---------------+-----------+------+
|1     |Smith   |1              |10         |3000  |
|1     |Smith   |1              |10         |3000  |
|2     |Rose    |1              |20         |4000  |
|3     |Williams|1              |10         |1000  |
|4     |Jones   |2              |10         |2000  |
|5     |Brown   |2              |40         |-1    |
|6     |Brown   |2              |50         |-1    |
+------+--------+---------------+-----------+------+



emp: Seq[(Int, String, Int, String, Int)] = List((1,Smith,1,10,3000), (1,Smith,1,10,3000), (2,Rose,1,20,4000), (3,Williams,1,10,1000), (4,Jones,2,10,2000), (5,Brown,2,40,-1), (6,Brown,2,50,-1))
empColumns: Seq[String] = List(emp_id, name, superior_emp_id, emp_dept_id, salary)
import spark.sqlContext.implicits._
datasetDF: org.apache.spark.sql.DataFrame = [emp_id: int, name: string ... 3 more fields]


In [2]:
datasetDF.show(5)

+------+--------+---------------+-----------+------+
|emp_id|    name|superior_emp_id|emp_dept_id|salary|
+------+--------+---------------+-----------+------+
|     1|   Smith|              1|         10|  3000|
|     1|   Smith|              1|         10|  3000|
|     2|    Rose|              1|         20|  4000|
|     3|Williams|              1|         10|  1000|
|     4|   Jones|              2|         10|  2000|
+------+--------+---------------+-----------+------+
only showing top 5 rows



In [3]:
val key_DF = Seq("name","superior_emp_id","emp_dept_id","salary")

val duplicateJoinDF = datasetDF.as("df1").join(datasetDF.as("df2"),
    key_DF,"left")
duplicateJoinDF.show(5)

+-----+---------------+-----------+------+------+------+
| name|superior_emp_id|emp_dept_id|salary|emp_id|emp_id|
+-----+---------------+-----------+------+------+------+
|Smith|              1|         10|  3000|     1|     1|
|Smith|              1|         10|  3000|     1|     1|
|Smith|              1|         10|  3000|     1|     1|
|Smith|              1|         10|  3000|     1|     1|
| Rose|              1|         20|  4000|     2|     2|
+-----+---------------+-----------+------+------+------+
only showing top 5 rows



key_DF: Seq[String] = List(name, superior_emp_id, emp_dept_id, salary)
duplicateJoinDF: org.apache.spark.sql.DataFrame = [name: string, superior_emp_id: int ... 4 more fields]


In [4]:
import scala.collection.mutable.ListBuffer

val list_single_colnames = ListBuffer[String]()
for (colname <- duplicateJoinDF.columns.toSeq){
    if (list_single_colnames.contains(colname)){
        list_single_colnames += colname.concat("_x") 
    }
    else{
        list_single_colnames += colname
    }
}
    

val duplicateJoinDF2 = duplicateJoinDF.toDF(list_single_colnames:_*)

import scala.collection.mutable.ListBuffer
list_single_colnames: scala.collection.mutable.ListBuffer[String] = ListBuffer(name, superior_emp_id, emp_dept_id, salary, emp_id, emp_id_x)
duplicateJoinDF2: org.apache.spark.sql.DataFrame = [name: string, superior_emp_id: int ... 4 more fields]


In [5]:
val key = "emp_id"
val identifiedDuplicatesDF = duplicateJoinDF2.groupBy(key,key + "_x").count()
  .filter($"count" > 1)
identifiedDuplicatesDF.show()

+------+--------+-----+
|emp_id|emp_id_x|count|
+------+--------+-----+
|     1|       1|    4|
+------+--------+-----+



key: String = emp_id
identifiedDuplicatesDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [emp_id: int, emp_id_x: int ... 1 more field]


In [6]:
val cols = identifiedDuplicatesDF.columns.toSeq

cols: Seq[String] = WrappedArray(emp_id, emp_id_x, count)


In [10]:
val duplicatesOutputDF = duplicateJoinDF2.as("df3").join(identifiedDuplicatesDF.as("df4"), col("df3." + key) === col("df4." + key + "_x")&&col("df3." + key) === col("df4." + key + "_x"),"inner").distinct

duplicatesOutputDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [name: string, superior_emp_id: int ... 7 more fields]


In [11]:
duplicatesOutputDF.distinct.show()

+-----+---------------+-----------+------+------+--------+------+--------+-----+
| name|superior_emp_id|emp_dept_id|salary|emp_id|emp_id_x|emp_id|emp_id_x|count|
+-----+---------------+-----------+------+------+--------+------+--------+-----+
|Smith|              1|         10|  3000|     1|       1|     1|       1|    4|
+-----+---------------+-----------+------+------+--------+------+--------+-----+



In [13]:
val nonDuplicatesDF = datasetDF.dropDuplicates(key_DF)
nonDuplicatesDF.show()

+------+--------+---------------+-----------+------+
|emp_id|    name|superior_emp_id|emp_dept_id|salary|
+------+--------+---------------+-----------+------+
|     1|   Smith|              1|         10|  3000|
|     2|    Rose|              1|         20|  4000|
|     3|Williams|              1|         10|  1000|
|     4|   Jones|              2|         10|  2000|
|     5|   Brown|              2|         40|    -1|
|     6|   Brown|              2|         50|    -1|
+------+--------+---------------+-----------+------+



nonDuplicatesDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [emp_id: int, name: string ... 3 more fields]
